<a href="https://colab.research.google.com/github/darknight11345/turtlebot4/blob/main/Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
pwd

'/content'

In [ ]:
#generate dataset
import random
import pandas as pd
import re

# Define a list of possible words for the sentence generation
actions = ["Deliver", "Go", "Take", "Bring", "Move", "Send", "Head towards", "Proceed", "Navigate", "Transport"]
locations = ["ICU", "reception", "emergency room", "doctors room", "nurse station"]  # Include 'nurse station' as a location
rooms = [str(i) for i in range(101, 601)]  # Random room numbers between 101 and 600
medicines = ["Paracetamol", "Aspirin", "Ibuprofen", "Tylenol", "Penicillin", "Antibiotics", "Cough Syrup"]

# List of multi-word locations to handle separately
multi_word_locations = ["nurse station", "doctors room", "emergency room"]

# Function to generate random sentences with action and location
def generate_sentence_with_action():
    action = random.choice(actions)
    medicine = random.choice(medicines)



    # Special logic for actions like 'Navigate', 'Proceed', 'Head towards'
    if action in ["Navigate", "Proceed", "Head towards", "Move", "Go"]:
        if random.choice([True, False]):
            # Location-based sentence
            location = random.choice(locations)
            sentence = f"{action} to {location} with {medicine}."
        else:
            # Room number-based sentence
            room = random.choice(rooms)
            sentence = f"{action} to Room {room} with {medicine}."

    else:
           # 50% chance for either a location or a room number
        if random.choice([True, False]):
            location = random.choice(locations)
            sentence = f"{action} {medicine} to {location}."
        else:
            room = random.choice(rooms)  # Random room number between 101 and 600
            sentence = f"{action} {medicine} to Room {room}."

    # Clean the sentence by removing unwanted special characters (e.g., periods, commas, etc.)
    sentence = re.sub(r'[^\w\s]', '', sentence)

    return sentence

# Generate 1000 sentences with actions, without sentence breaks
data = []
sentence_id = 1
for _ in range(1000):
    sentence = generate_sentence_with_action()

    # Handle multi-word locations before splitting
    for multi_word_loc in multi_word_locations:
        if multi_word_loc in sentence:
            sentence = sentence.replace(multi_word_loc, f"{multi_word_loc.replace(' ', '_')}")

    tokens = sentence.split()

    # Process each token in the sentence
    i = 0
    while i < len(tokens):
        token = tokens[i]

        # Special handling for "Head towards" as a single ACTION entity
        if token == "Head" and i + 1 < len(tokens) and tokens[i + 1] == "towards":
            data.append((sentence_id, "Head towards", "ACTION"))
            i += 1  # Skip the next token ("towards")
        elif token in actions and token != "Head":
            data.append((sentence_id, token, "ACTION"))
        elif token in medicines:
            data.append((sentence_id, token, "MEDICINE"))
        elif token == "Room":
            # Combine "Room" and the next token (the number) as a LOCATION entity
            if i + 1 < len(tokens):
                room_number = f"Room {tokens[i+1]}"
                data.append((sentence_id, room_number, "LOCATION"))
                i += 1  # Skip the next token (room number)
        elif token.replace("_", " ") in multi_word_locations:
            # Replace underscores back to spaces for multi-word locations
            location = token.replace("_", " ")
            data.append((sentence_id, location, "LOCATION"))
        elif token in locations:
            data.append((sentence_id, token, "LOCATION"))  # Locations like ICU, reception, etc.
        else:
            data.append((sentence_id, token, "O"))  # Words that are not part of any entity

        i += 1

    sentence_id += 1

# Create DataFrame and save the dataset to CSV
df = pd.DataFrame(data, columns=["Sentence_ID", "Word", "Label"])
df.to_csv("ner_dataset_with_head_towards_handling.csv", index=False)

print("CSV file 'ner_dataset_with_head_towards_handling.csv' created successfully.")


In [ ]:
#setup of dataset

In [3]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00
   ━━

In [4]:
!pip install fsspec==2024.10.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [5]:
import pandas as pd
data = pd.read_csv("ner_dataset_with_head_towards_handling.csv",encoding="latin1" )

In [6]:
data.head(30)

,Sentence_ID,Word,Label
0,1,Deliver,ACTION
1,1,Tylenol,MEDICINE
2,1,to,O
3,1,Room 493,LOCATION
4,2,Go,ACTION
5,2,to,O
6,2,ICU,LOCATION
7,2,with,O
8,2,Aspirin,MEDICINE
9,3,Go,ACTION


In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [8]:
data.rename(columns={"Sentence_ID":"sentence_id","Word":"words","Label":"labels"}, inplace =True)

In [9]:
data["labels"] = data["labels"].str.upper()

In [10]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [12]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [14]:
test_data

,sentence_id,words,labels
3678,787,with,O
1348,290,Send,ACTION
4163,891,with,O
2772,592,Move,ACTION
3643,779,with,O
...,...,...,...
2631,561,to,O
2676,571,Room 220,LOCATION
1830,391,Take,ACTION
2636,562,to,O


In [15]:
train_data

,sentence_id,words,labels
289,62,with,O
3020,646,Head towards,ACTION
1436,308,Aspirin,MEDICINE
3038,649,Room 346,LOCATION
4457,954,Syrup,O
...,...,...,...
170,38,Head towards,ACTION
2264,483,to,O
2977,636,Room 102,LOCATION
878,188,to,O


In [16]:
#Model Training

In [13]:

!pip install torch  --index-url https://download.pytorch.org/whl/cu124


Looking in indexes: https://download.pytorch.org/whl/cu124


In [14]:


from simpletransformers.ner import NERModel,NERArgs

In [15]:
label = data["labels"].unique().tolist()
label

['ACTION', 'MEDICINE', 'O', 'LOCATION']

In [26]:
args = NERArgs()
args.num_train_epochs = 5
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32
args.gradient_accumulation_steps = 4  # Adjust this to your needs
args.max_seq_length = 128  # Adjust this according to your data



In [27]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
#model.train_model(train_data,eval_data = test_data,acc=accuracy_score)
output_dir = "./best_model"
model.train_model(train_data, eval_data=test_data, acc=accuracy_score, save_best_model=True, output_dir=output_dir)

model.save_model("./text_trained_model_bert")

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/32 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/32 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
import torch

# Save the model's state_dict (weights only)
torch.save(model.model.state_dict(), "model_weights.pth")

In [30]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/20 [00:00<?, ?it/s]

In [34]:
result

{'eval_loss': 0.00030960602016421034,
 'precision': 1.0,
 'recall': 1.0,
 'f1_score': 1.0}

In [35]:
prediction, model_output = model.predict(["deploy tablets to 102 room"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
prediction

[[{'deploy': 'ACTION'},
  {'tablets': 'LOCATION'},
  {'to': 'O'},
  {'102': 'LOCATION'},
  {'room': 'LOCATION'}]]

In [37]:
model_output

[[{'deploy': [[6.7146554, -2.0406265, -1.9777999, -2.4910898]]},
  {'tablets': [[-1.938357, -0.12115304, -2.6209147, 4.6842175]]},
  {'to': [[-2.8898273, -2.3097792, 6.8954353, -2.305391]]},
  {'102': [[-2.2250614, -1.6113223, -1.7799599, 4.550294]]},
  {'room': [[-2.5461636, -1.8109851, -2.2558794, 6.3508377]]}]]

In [38]:
#inference

# Load the best saved model for inference
#model1 = NERModel("bert", "./best_model", use_cuda=False)


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory ./best_model.

In [39]:
model1 = NERModel("bert", "./outputs", use_cuda=False)

In [45]:
prediction1, model_output = model1.predict(["deliver paracetamol to 102 room"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
prediction1

[[{'deliver': 'ACTION'},
  {'paracetamol': 'MEDICINE'},
  {'to': 'O'},
  {'102': 'LOCATION'},
  {'room': 'LOCATION'}]]